# Homework 10: Cleaning data with Regular Expressions

Time to use regular expressions!

# Hints and notes

### Opening files in subdirectories

Notice that this notebook might be **homework/**, but!!! the csvs and text files might be in **homework/scraped/** or **/homework/scraped/minutes_pdfs** or **/homework/pdfs/**. To open a file in a subdirectory, instead of having the filename be `"file.csv"` you'll just use `"some/subfolder/file.csv"`

### Opening text files

This will open up a file, read it in and show you the first 500 characters.

```python
contents = open("your-filename.txt").read()
contents[0:500]
```

> You might need `open("your-filename.txt", encoding="utf8").read()`

### Using regex

For some dumb reason you need to put `r` in front of the string you use when you're talking about regex. Just plain `"(\d\d\d)"` will usually work, but *sometimes* it won't and you'll need `r"(\d\d\d)`. It's best to just use the `r` all of the time, if you can remember!

### Using `.str.extract`

When you use `.str.extract`, you're always going to **capture one thing** and save it to a new column. You need to wrap the things you're interested in with parenthesis `(` `)`.

```python
df['phone_number'] = df['old_column'].str.extract(r"My phone number is (\d\d\d-\d\d\d-\d\d\d\d)")
```

### Setting pandas options

Pandas has a lot of options, like how many columns or rows it will show you, or how many characters it will show in a column before it stops showing you anything. Here are a few useful ones:

* `display.max_cols`: Number of columns to show at once
* `display.max_rows`: Number of rows to show at once
* `display.max_colwidth`: Maximum number of characters displayed from a string

You can set them using `pd.set_option("display.max_rows", 1000)`, for example, to show 1000 rows at a time. You can find a lot more at https://pandas.pydata.org/pandas-docs/stable/generated/pandas.set_option.html

### Regular expressions reference

I personally think http://www.regular-expressions.info/ is a wonderful wonderful reference (and tutorial), even if it's ugly! But here's a quick reference for you:

* `\d` is a digit
* `\d*` is zero or more digits 
* `\d+` is one or more digits
* `.` matches anything for ONE character
* `.*` is "give me anything forever"
* `\s` is whitespace, a.k.a. spaces and tabs
* `\w` is a word character, which includes capital and lowercase letters, numbers and hyphens.
* You can put `*` after anything, so `\w*` would mean "as many word characters as you can find"
* `\b` is a word boundary (you'll need the `r""` thing for this one)
* `( )` is a "capture group" for saving something
* `\1` is used when doing find/replace to say "put the first captured group here" (note, it's a dollar sign instead of a backslash in some editors)
* `[ABCDE]` is a character class, which means "match one of these, I don't care which"
* dollar sign means "end of the line"
* caret ^ means "beginning of the line"
* `\.` means "no really seriously I mean a period not just anything"
* You can use `\` with anything else that would normally be a special character, too, not just periods. `(` or `[` or whatever.

### Cleaning up extracted columns

Sometimes you get `\n` (newlines) or spaces or `\t` (tabs) or stuff at the beginning or the end of your column. `.str.strip()` will usually take care of that, just attach it after your `.str.extract()`

After you extract something, it's still a string even though you look at it and know it's a number. Use `.astype(int)` to turn it into an integer (no decimal) or `.astype(float)` to turn it into a float (yes decimal)

### Writing regular expressions in general

Even if I'm using regex in pandas or Python, I like to test them in my text editor with "Find." The highlighting really helps me see if I'm matching things! I also like to think "what stays the same?" when designing patterns, write those parts first, then fill in the blanks with what I want to capture.

## Importing

There might be more, I just wanted to put this up here for the `pd.set_option` part. It allows you to see a lot of content in a single column of pandas, which will be important for some parts below.

In [1]:
import re
import pandas as pd
pd.set_option('display.max_colwidth', 500)

# Part 1: Using `.str.extract` to pull data from columns in pandas

## 1.1 H&M

Open up `hm.csv` from the `scraped` directory. I want **four new columns**:

1. `price_original`, the original price, one of the new price
2. `price_discounted`, the discounted price
3. `pct_discount`, the percent discount
4. `article_id`, the article id (from the url)

Save as **hm_cleaned.csv**.

**Note:** When you look at it, it... won't look right. I don't know why, pandas is weird. Look at the `price` column by itself using `df['price']` before you write your regex.

**Tip:** Remember that `$` is a special regex symbol! You might need to escape it.

**Tip:** When doing `.str.extract`, the whole match doesn't get captured, only what you put `()` around! Think about anchoring to different points of the string, or things in the string.

**Tip:** Not all prices have cents!

**Tip:** Your first instinct about how to compute the percent discount is probably wrong

In [43]:
df = pd.read_csv("hm.csv")
df.price

0       $59.99 $129
1      $6.99 $17.99
2      $7.99 $17.99
3      $6.99 $12.99
4     $17.99 $24.99
5       $3.99 $9.99
6      $7.99 $17.99
7      $7.99 $17.99
8      $9.99 $17.99
9     $27.99 $34.99
10      $2.99 $5.99
11      $2.99 $5.99
12     $9.99 $12.99
13    $39.99 $49.99
14    $39.99 $49.99
15     $7.99 $17.99
16    $19.99 $29.99
17      $2.99 $5.99
18     $7.99 $17.99
19    $14.99 $24.99
20     $9.99 $24.99
21     $7.99 $12.99
22     $7.99 $17.99
23      $4.99 $9.99
24     $9.99 $12.99
25     $7.99 $17.99
26      $3.99 $6.99
27     $7.99 $17.99
28      $2.99 $6.99
29     $9.99 $12.99
30    $14.99 $34.99
31     $7.99 $17.99
32    $24.99 $49.99
33    $29.99 $79.99
34     $4.99 $12.99
35    $19.99 $24.99
36      $2.99 $5.99
37    $14.99 $34.99
38     $6.99 $17.99
39     $6.99 $17.99
40     $6.99 $17.99
41         $99 $199
42       $54.99 $99
43     $6.99 $17.99
44     $4.99 $12.99
45     $7.99 $17.99
46      $3.99 $9.99
47     $6.99 $17.99
48     $4.99 $12.99
49      $2.99 $5.99


In [72]:
df['price_discounted'] = df['price'].str.extract(r"\$(\d+.\d\d) ")
df['price_discounted'].head()

0    59.99
1     6.99
2     7.99
3     6.99
4    17.99
Name: price_discounted, dtype: object

In [52]:
df['price_original'] = df.price.str.extract(r" \$(\d*\d*.\d\d)")
df['price_original'].head()

0      129
1    17.99
2    17.99
3    12.99
4    24.99
Name: price_original, dtype: object

In [55]:
df['article_id'] = df.url.str.extract(r"=(.*)$")
df['article_id'].head()

0    13472-N
1    35079-D
2    72462-A
3    58926-C
4    74242-A
Name: article_id, dtype: object

In [88]:
df['price_discounted'] = df['price_discounted'].astype(float)
df['price_original'] = df['price_original'].astype(float)
df.dtypes

df['pct_discount'] = round((df['price_original'] - df['price_discounted']) / df['price_original'] * 100, 2)
df['pct_discount'].head()

0    53.50
1    61.15
2    55.59
3    46.19
4    28.01
Name: pct_discount, dtype: float64

In [85]:
df.head()

,name,price,url,price_discounted,price_original,article_id,pct_discount
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,59.99,129.00,13472-N,53.50
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,6.99,17.99,35079-D,61.15
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,7.99,17.99,72462-A,55.59
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,6.99,12.99,58926-C,46.19
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,17.99,24.99,74242-A,28.01


## 1.2 Sci-Fi Authors

Open up `sci-fi.csv` to clean. Get rid of the `\n` on the title and and give me six new columns:

* `avg_rating`
* `rating_count`
* `total_score`
* `score_votes`
* `series` the series the book belongs to
* `series_no` the book in the series that it is

For series, I'm talking about e.g. `(The Hunger Games, #1)` is `series` "The Hunter Games" and `series_no` 1.

Save as **sci-fi_cleaned.csv**.

**Tip:** You don't need regex to clean the title - there's a special thing that removes whitespace from the beginning/end of strings

**Tip:** Remember that `(` and `)` are special characters

**BONUS:** When you make the `total_score` column, pay close attention to it. If you notice the problem, fix it.

**BONUS:** You don't need these columns to be numbers, but life would be better if they were. 

In [90]:
df = pd.read_csv("sci-fi.csv")
df.head()

,full_rating,full_score,rank,title,url
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,\nThe Handmaid's Tale\n,/book/show/38447.The_Handmaid_s_Tale
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"\nThe Hunger Games (The Hunger Games, #1)\n",/book/show/2767052-the-hunger-games
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"\nFrankenstein, or The Modern Prometheus\n",/book/show/18490.Frankenstein_or_The_Modern_Prometheus
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"\nA Wrinkle in Time (A Wrinkle in Time Quintet, #1)\n",/book/show/18131.A_Wrinkle_in_Time
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,\nThe Left Hand of Darkness\n,/book/show/18423.The_Left_Hand_of_Darkness


In [94]:
df['title'] = df.title.str.strip()
df.title.head()

0                                  The Handmaid's Tale
1              The Hunger Games (The Hunger Games, #1)
2               Frankenstein, or The Modern Prometheus
3    A Wrinkle in Time (A Wrinkle in Time Quintet, #1)
4                            The Left Hand of Darkness
Name: title, dtype: object

In [106]:
df['avg_rating'] = df.full_rating.str.extract(r"(\d.\d\d )")
df['avg_rating'].head()

0    4.07 
1    4.34 
2    3.76 
3    4.04 
4    4.06 
Name: avg_rating, dtype: object

In [107]:
df['rating_count'] = df.full_rating.str.extract(r" (\d*,*\d*,*\d*) ratings$")
df['rating_count'].head()

0      785,502
1    5,212,935
2      922,308
3      702,272
4       77,664
Name: rating_count, dtype: object

In [117]:
df['total_score'] = df.full_score.str.extract(r"score: (\d*,*\d*,*\d*),")
df['total_score'] = df['total_score'].str.replace(",", "").apply(pd.to_numeric)
df.total_score.head()

0    28539
1    27566
2    20049
3    17684
4    16070
Name: total_score, dtype: int64

In [123]:
df['score_votes'] = df.full_score.str.extract(r"(\d*\d*\d*) people voted")
df.score_votes.head()

0    292
1    282
2    205
3    185
4    165
Name: score_votes, dtype: object

In [125]:
df['series'] = df.title.str.extract(r"\((.*),.*\)")
df.series.head()

0                          NaN
1             The Hunger Games
2                          NaN
3    A Wrinkle in Time Quintet
4                          NaN
Name: series, dtype: object

In [126]:
df['series_no'] = df.title.str.extract(r"\(.*,(.*)\)")
df.series_no.head()

0    NaN
1     #1
2    NaN
3     #1
4    NaN
Name: series_no, dtype: object

In [127]:
df.head()

,full_rating,full_score,rank,title,url,avg_rating,rating_count,total_score,score_votes,series,series_no
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,4.07,"785,502",28539,292,NaN,NaN
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,4.34,"5,212,935",27566,282,The Hunger Games,#1
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,3.76,"922,308",20049,205,NaN,NaN
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,4.04,"702,272",17684,185,A Wrinkle in Time Quintet,#1
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,4.06,"77,664",16070,165,NaN,NaN


## 1.3 Where you're just doing one of my former students' projects

Once upon a time my student Stefan did a project that involved some lawyer stuff. Most of the content was in PDFs, though! I converted them to text files and put them into the `pdfs` folder, and gave you code below to open up each of them and save their contents into a dataframe.

What a nice dataframe! I want you to add the following columns to it:

* `lawyer_app`, the applicant's lawyer (pro se means that they did it themselves, that's fine)
* `lawyer_gov`, the government's lawyer
* `judge`, the name of the judge
* `access`, whether the clearance is granted or denied (although you might miss a few)

Save as **court_cleaned.csv**.

**Note:** You can look at the original PDFs, they're also included.

**Note:** This uses a fun utility called `glob`, which is mostly fun because you use it as `glob.glob`. It's used to find files that match a certain filename pattern.

**BONUS:** You'll be happy once you get the judge, but make sure it doesn't have any extra punctuation on it.

**BONUS:** You can for some words using `.str.contains("blah")` and save it into new columns. Maybe `has_debt`, `has_bankruptcy`, etc.

> It's okay if it isn't perfect. Converting PDF into data rarely is! Usually you get 90% of it done with computers, then send people to enter the other 10% by hand.

In [153]:
import glob
filenames = glob.glob("pdfs/*.txt")
contents = [open(filename, encoding="utf8").read() for filename in filenames]
df = pd.DataFrame({'filename': filenames, 'content': contents})

In [156]:
df.head(4)

,filename,content
0,pdfs\11-02438.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-02438 \n\nFor Government: Stephanie C. Hess, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\..."
1,pdfs\11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n\n \n\nISCR Case No. 11-03073 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\nFor Government: Robert J. Kilmartin, Esq., Department Counse..."
2,pdfs\11-04909.h1.pdf.txt,"\n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n DEPARTMENT OF DEFENSE \n\n \n \n\n \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-04909 \n\n \n\nFor Government: Richard Stevens, Esq., Department Counsel \n\nFor Applicant: Pro se \n\n \n\n \nDUFFY, James F., Administrative Judge: \n\n \nApplicant mitigated \n\nconsiderations). Clearance is ..."
3,pdfs\11-07728.h1.pdf.txt,"DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n------------------------ \n \n\n \n\nISCR Case No. 11-07728 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n\n \n \n\nAppearances \n\n___________ \n\n \nDecision \n\n___________ \n\nFor Government: Julie R. Mendez, Esq., Department Counsel \n\nFor Applicant: Mark S. Zaid, Esq. \n\n \n\nHARVEY, Mark, Administrative Judge: \n \..."


Okay, now do the work and **make those new columns!**

In [158]:
df['lawyer_app'] = df.content.str.extract(r"For Applicant[:]*[\n ]*(.*).*", re.IGNORECASE)  
df['lawyer_app']

0                      Pro se 
1          Mark S. Zaid, Esq. 
2                      Pro se 
3          Mark S. Zaid, Esq. 
4                      Pro se 
5                      Pro se 
6                       Pro se
7     Ryan C. Nerney, Esquire 
8                      Pro se 
9                      Pro se 
10                     Pro Se 
11                      Pro se
12                      Pro se
13     Stephen Glassman, Esq. 
14                     Pro se 
Name: lawyer_app, dtype: object

In [159]:
df['lawyer_gov'] = df.content.str.extract(r"For Government[:]*[\n ]*(.*), Esq\w*", re.IGNORECASE)  
df.head()

,filename,content,lawyer_app,lawyer_gov
0,pdfs\11-02438.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-02438 \n\nFor Government: Stephanie C. Hess, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\...",Pro se,Stephanie C. Hess
1,pdfs\11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n\n \n\nISCR Case No. 11-03073 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\nFor Government: Robert J. Kilmartin, Esq., Department Counse...","Mark S. Zaid, Esq.",Robert J. Kilmartin
2,pdfs\11-04909.h1.pdf.txt,"\n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n DEPARTMENT OF DEFENSE \n\n \n \n\n \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-04909 \n\n \n\nFor Government: Richard Stevens, Esq., Department Counsel \n\nFor Applicant: Pro se \n\n \n\n \nDUFFY, James F., Administrative Judge: \n\n \nApplicant mitigated \n\nconsiderations). Clearance is ...",Pro se,Richard Stevens
3,pdfs\11-07728.h1.pdf.txt,"DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n------------------------ \n \n\n \n\nISCR Case No. 11-07728 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n\n \n \n\nAppearances \n\n___________ \n\n \nDecision \n\n___________ \n\nFor Government: Julie R. Mendez, Esq., Department Counsel \n\nFor Applicant: Mark S. Zaid, Esq. \n\n \n\nHARVEY, Mark, Administrative Judge: \n \...","Mark S. Zaid, Esq.",Julie R. Mendez
4,pdfs\11-08313.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n\nISCR Case No. 11-08313 \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n--------------- \n \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\nFor Government: Julie R. Mendez, Esquire, Department Counsel \n\nFo...",Pro se,Julie R. Mendez


In [160]:
df['judge'] = df.content.str.extract(r"\n(.*)Administrative Judge:", re.IGNORECASE)  
df.head()

,filename,content,lawyer_app,lawyer_gov,judge
0,pdfs\11-02438.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-02438 \n\nFor Government: Stephanie C. Hess, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\...",Pro se,Stephanie C. Hess,"COACHER, Robert E.,"
1,pdfs\11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n\n \n\nISCR Case No. 11-03073 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\nFor Government: Robert J. Kilmartin, Esq., Department Counse...","Mark S. Zaid, Esq.",Robert J. Kilmartin,"LOUGHRAN, Edward W.,"
2,pdfs\11-04909.h1.pdf.txt,"\n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n DEPARTMENT OF DEFENSE \n\n \n \n\n \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-04909 \n\n \n\nFor Government: Richard Stevens, Esq., Department Counsel \n\nFor Applicant: Pro se \n\n \n\n \nDUFFY, James F., Administrative Judge: \n\n \nApplicant mitigated \n\nconsiderations). Clearance is ...",Pro se,Richard Stevens,"DUFFY, James F.,"
3,pdfs\11-07728.h1.pdf.txt,"DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n------------------------ \n \n\n \n\nISCR Case No. 11-07728 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n\n \n \n\nAppearances \n\n___________ \n\n \nDecision \n\n___________ \n\nFor Government: Julie R. Mendez, Esq., Department Counsel \n\nFor Applicant: Mark S. Zaid, Esq. \n\n \n\nHARVEY, Mark, Administrative Judge: \n \...","Mark S. Zaid, Esq.",Julie R. Mendez,"HARVEY, Mark,"
4,pdfs\11-08313.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n\nISCR Case No. 11-08313 \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n--------------- \n \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\nFor Government: Julie R. Mendez, Esquire, Department Counsel \n\nFo...",Pro se,Julie R. Mendez,"MARSHALL, Jr., Arthur E.,"


In [161]:
df['access'] = df.content.str.extract(r"[access|clearance] .* (denied|granted)", re.IGNORECASE)  
df.head()

,filename,content,lawyer_app,lawyer_gov,judge,access
0,pdfs\11-02438.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-02438 \n\nFor Government: Stephanie C. Hess, Esq., Department Counsel \n\nFor Applicant: Pro se \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\...",Pro se,Stephanie C. Hess,"COACHER, Robert E.,",denied
1,pdfs\11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n\n \n\nISCR Case No. 11-03073 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\nFor Government: Robert J. Kilmartin, Esq., Department Counse...","Mark S. Zaid, Esq.",Robert J. Kilmartin,"LOUGHRAN, Edward W.,",granted
2,pdfs\11-04909.h1.pdf.txt,"\n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n DEPARTMENT OF DEFENSE \n\n \n \n\n \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-04909 \n\n \n\nFor Government: Richard Stevens, Esq., Department Counsel \n\nFor Applicant: Pro se \n\n \n\n \nDUFFY, James F., Administrative Judge: \n\n \nApplicant mitigated \n\nconsiderations). Clearance is ...",Pro se,Richard Stevens,"DUFFY, James F.,",granted
3,pdfs\11-07728.h1.pdf.txt,"DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n------------------------ \n \n\n \n\nISCR Case No. 11-07728 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n\n \n \n\nAppearances \n\n___________ \n\n \nDecision \n\n___________ \n\nFor Government: Julie R. Mendez, Esq., Department Counsel \n\nFor Applicant: Mark S. Zaid, Esq. \n\n \n\nHARVEY, Mark, Administrative Judge: \n \...","Mark S. Zaid, Esq.",Julie R. Mendez,"HARVEY, Mark,",granted
4,pdfs\11-08313.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n\nISCR Case No. 11-08313 \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n--------------- \n \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\nFor Government: Julie R. Mendez, Esquire, Department Counsel \n\nFo...",Pro se,Julie R. Mendez,"MARSHALL, Jr., Arthur E.,",denied


# Reading books

When you're doing text work, you're legally obligated work on Jane Austen's Pride and Prejudice (at least I *think* so). Let's do some naive analysis of it!

## Read in Jane Austen's Pride and Prejudice (without moving the file!)

It's in the `data/` directory, and named `Austen_Pride.txt`.

In [162]:
contents = open("data/Austen_Pride.txt").read()

## Look at the first 500 or so characters of it 

In [164]:
contents[0:500]

' Pride and Prejudice\nby Jane Austen\nChapter 1\nIt is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.\nHowever little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.\n"My dear Mr. Bennet," said his lady to him one day, "have you heard that Nethe'

## Use a regular expression to find every "he" or "she" in the book. There should be about 3000 of them.

**Tip:** Do you know about **word boundaries?** `\b` means "the beginning of end of a word."

**Tip:** You might also want to use `re.IGNORECASE`. Maybe you'll need to google it? 

**Tip:** Do NOT use `re.compile`

In [165]:
len(re.findall(r"[\n ]+he\b|[\n ]+she\b", contents, re.IGNORECASE))

2990

## Use a regular expression to find those same "he" or "she"s, but also match *the word after it*

The first four should be:

* he is
* he had
* she told
* he came

In [176]:
he_she = re.findall(r"[\n ]+he\b \w*.|[\n ]+she\b \w*.", contents, re.IGNORECASE)
he_she

[' he is ',
 ' he had ',
 ' she told ',
 ' he came ',
 ' he agreed ',
 ' he is ',
 ' he married ',
 ' he may ',
 ' he comes.',
 ' she ought ',
 ' he comes ',
 ' he chooses ',
 ' she is ',
 ' She was ',
 ' she was ',
 ' she fancied ',
 ' He had ',
 ' he should ',
 ' she had ',
 ' he suddenly ',
 ' She has ',
 ' She is ',
 ' she will ',
 ' she will ',
 ' he continued,',
 ' he wished;',
 ' she began ',
 ' she had ',
 ' he spoke,',
 ' he left ',
 ' he would ',
 ' he eluded ',
 ' He was ',
 ' he meant ',
 ' He had ',
 ' he had ',
 ' he saw ',
 ' he wore ',
 ' She could ',
 ' he could ',
 ' she began ',
 ' he might ',
 ' he ought ',
 ' he brought ',
 ' he had ',
 ' he was ',
 ' he was ',
 ' he was ',
 ' he was ',
 ' He was ',
 ' he would ',
 ' She is ',
 ' he looked ',
 ' he withdrew ',
 ' She told ',
 ' she had ',
 ' she had ',
 ' he was ',
 ' he had ',
 ' He had ',
 ' he had ',
 ' she entered ',
 ' she looked;',
 ' he actually ',
 ' she was ',
 ' he asked ',
 ' he asked ',
 ' he did ',
 ' 

## Use capture groups to save the pronoun (he/she) as one match and the word as another

The first five should look like

```
[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed')]```

In [177]:
heshe = re.findall(r"[\n ]+(he*|she*) (\w*)", contents, re.IGNORECASE)
heshe

[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed'),
 ('he', 'is'),
 ('he', 'married'),
 ('he', 'may'),
 ('he', 'comes'),
 ('she', 'ought'),
 ('he', 'comes'),
 ('he', 'chooses'),
 ('she', 'is'),
 ('She', 'was'),
 ('she', 'was'),
 ('she', 'fancied'),
 ('He', 'had'),
 ('he', 'should'),
 ('she', 'had'),
 ('he', 'suddenly'),
 ('She', 'has'),
 ('She', 'is'),
 ('she', 'will'),
 ('she', 'will'),
 ('he', 'continued'),
 ('he', 'wished'),
 ('she', 'began'),
 ('she', 'had'),
 ('he', 'spoke'),
 ('he', 'left'),
 ('he', 'would'),
 ('he', 'eluded'),
 ('He', 'was'),
 ('he', 'meant'),
 ('He', 'had'),
 ('he', 'had'),
 ('he', 'saw'),
 ('he', 'wore'),
 ('She', 'could'),
 ('he', 'could'),
 ('she', 'began'),
 ('he', 'might'),
 ('he', 'ought'),
 ('he', 'brought'),
 ('he', 'had'),
 ('he', 'was'),
 ('he', 'was'),
 ('he', 'was'),
 ('he', 'was'),
 ('He', 'was'),
 ('he', 'would'),
 ('She', 'is'),
 ('he', 'looked'),
 ('he', 'withdrew'),
 ('She', 'told'),
 ('she', 'had'),
 ('she', 'h

## Save those matches into a dataframe

You can give the column names with `columns=['pronoun', 'verb']`

In [178]:
df = pd.DataFrame(heshe, columns=['pronoun', 'verb'])
df.head()

,pronoun,verb
0,he,is
1,he,had
2,she,told
3,he,came
4,he,agreed


## How many times is each pronoun used?

In [179]:
df.pronoun.value_counts()

she    1316
he     1048
She     306
He      212
Name: pronoun, dtype: int64

## Oh, wait, clean that up.

Make it only 'he' and 'she' lowercase.

It should be about 1600 'she' and 1300 'he'

In [181]:
df['pronoun'] = df.pronoun.str.lower()
df.pronoun.value_counts()

she    1622
he     1260
Name: pronoun, dtype: int64

## What are the top 20 most common verbs?

In [185]:
df.verb.value_counts().head(20)

was          371
had          369
could        170
is           116
would         92
has           65
did           64
will          49
might         46
should        41
felt          37
must          36
said          33
saw           31
thought       31
added         31
then          26
replied       22
continued     21
looked        20
Name: verb, dtype: int64

## What are the top 20 most common verbs for 'he', and the top 20 most common for 'she'

**Tip:** Don't use groupby, just filter. If you want to know how, though, you can also look at "value counts for different categories" on [this page](http://jonathansoma.com/lede/foundations-2017/classes/more-pandas/class-notes/)

In [186]:
verb_he = df[df['pronoun']=="he"]
verb_he.verb.value_counts().head(20)

had          165
was          159
is            62
has           47
could         39
would         34
did           28
should        26
will          24
must          23
might         21
replied       14
thought       13
said          13
came          13
may           12
never         12
then          11
does          11
continued     10
Name: verb, dtype: int64

In [187]:
verb_she = df[df['pronoun']=="she"]
verb_she.verb.value_counts().head(20)

was        212
had        204
could      131
would       58
is          54
did         36
felt        32
saw         28
will        25
might       25
added       23
said        20
has         18
thought     18
should      15
then        15
must        13
found       13
soon        12
spoke       11
Name: verb, dtype: int64

## Who cries more, men or women? Give me a percentage answer.

**Tip:** It's `cried`, because of, you know, how books are written

In [196]:
she_cries = round(verb_she.verb.value_counts(normalize=True).loc['cried'] *100,2)
he_cries = round(verb_he.verb.value_counts(normalize=True).loc['cried'] *100,2)
print(she_cries)
print(he_cries)

0.68
0.08


## How much more common is 'he' than 'she' in J.R.R. Tolkein's Fellowship of the Ring? How does that compare to Pride and Prejudice?

The book is in the same directory.

In [201]:
contents = open("data/Lord of the Rings - 01 - The Fellowship of the Ring - J. R. R. Tolkien - 1955.txt", encoding="utf8").read()

In [202]:
contents[0:500]

'THE FELLOWSHIP OF THE RING\n\n\n\n\nBEING THE FIRST PART OF\n\nTHE LORD OF THE RINGS\n\nBY\n\nJ.R.R. TOLKIEN\n\n\n\n\n\nThree Rings for the Elven-kings under the sky,\n\nSeven for the Dwarf-lords in their halls of stone,\n\nNine for Mortal Men doomed to die,\n\nOne for the Dark Lord on his dark throne\n\nIn the Land of Mordor where the Shadows lie.\n\nOne Ring to rule them all, One Ring to find them,\n\nOne Ring to bring them all and in the darkness bind them\n\nIn the Land of Mordor where the Shadows lie.\n\n\n\n\n\nCONTENTS\n\n\nCOV'

In [203]:
df = pd.DataFrame(re.findall(r"[\n ]+he\b|[\n ]+she\b", contents, re.IGNORECASE), columns=["pronoun"])
df['pronoun']= df.pronoun.str.lower()
df['pronoun']= df.pronoun.str.strip()

df.pronoun.value_counts("normalize")*100

he     95.047619
she     4.952381
Name: pronoun, dtype: float64